# 4. Building models

pyMC를 이용해 베이지안을 계산하는 데 약 3가지 종류의 데이터들이 있습니다. 
Stochastic, Deterministic and Potential. (+container) 

이 장에서는 각 종류의 데이터가 어떤 식으로 쓸 수 있는지 (데이터 스트럭쳐에서 나오는 용어..겠죠?) 
파이썬에서 구현되는 방식을 설명하고 있습니다. 


## 4.1. The Stochastic class

### 4.1.1. Creation of stochastic variables
 먼저 데이터의 성질(uniform, poisson... etc) 을 설정한 뒤에, 그 조건에 맞는 '랜덤' (완전 랜덤은 아닌거죠) 데이터들을 만들어주는 방법입니다. 

1) Automatic 
서브 클래스로 각 타입이 지정되어 있어서 한 줄 코딩으로 생성 가능하다고 합니다. 예를 들면, 지난 튜토리얼에서 uniformly-distributed discrete variable로 지정되었던 switchpoint 의 경우:

In [ ]:
switchpoint = DiscreteUniform('switchpoint', lower=0, upper=110, doc='Switchpoint[year]')

SciPy 가 설치 되어 있는 경우에는 PyMC에 없는 Stochastic subclasses 나 아예 지정되지 않은 경우에도 지정해서 사용이 가능하다고 합니다. 

2) Decorator

 아예 처음부터 log-probability 에 대한 값으로 데이터 처리를 해주는 경우도 있습니다. (데코레이터는.. 함수에 연산 한번 더해주기?이므로 Uniform log-likelihood에 대한 stochastic을 만들어주는 코드..인 것 같습니다~ "Generally, decorators enhance functions with additional properties or functionality.") 

In [ ]:
@pymc.stochastic(dtype=int)
def switchpoint(value=1900, t_l=1851, t_h=1962):
    """The switchpoint for the rate of disaster occurrence."""
    if value > t_h or value < t_l:
        # Invalid values
        return -np.inf
    else:
        # Uniform log-likelihood
        return -np.log(t_h - t_l + 1)

- @stochastic 에 영향을 받는 함수는 반드시 -log 로 정의 되어야 합니다.  
- PyMC and SciPy both provide implementations of several standard probability distributions that may be helpful for creating custom stochastic variables.




In [ ]:
@pymc.stochastic(dtype=int)
def switchpoint(value=1900, t_l=1851, t_h=1962):
    """The switchpoint for the rate of disaster occurrence."""

    def logp(value, t_l, t_h):
        if value > t_h or value < t_l:
            return -np.inf
        else:
            return -np.log(t_h - t_l + 1)

    def random(t_l, t_h):
        from numpy.random import random
        return np.round( (t_l - t_h) * random() ) + t_l




3) Direct
It’s possible to instantiate Stochastic directly:

In [ ]:
def switchpoint_logp(value, t_l, t_h):
    if value > t_h or value < t_l:
        return -np.inf
    else:
        return -np.log(t_h - t_l + 1)

def switchpoint_rand(t_l, t_h):
    from numpy.random import random
    return np.round( (t_l - t_h) * random() ) + t_l


switchpoint = Stochastic( logp = switchpoint_logp,
                doc = 'The switchpoint for the rate of disaster occurrence.',
                name = 'switchpoint',
                parents = {'t_l': 1851, 't_h': 1962},
                random = switchpoint_rand,
                trace = True,
                value = 1900,
                dtype=int,
                rseed = 1.,
                observed = False,
                cache_depth = 2,
                plot=True,
                verbose = 0)


Notice that the log-probability and random variate functions are specified externally and passed to Stochastic as arguments. This is a rather awkward way to instantiate a stochastic variable; consequently, such implementations should be rare.

경고 : 변수 값을 in-place로 바꾸면 안됩니다.
A Warning: Don’t update stochastic variables’ values in-place

in-place 란?
A.value = new_value
The following are in-place updates and should _never_ be used:

* ``A.value += 3``
* ``A.value[2,1] = 5``
* ``A.value.attribute = new_attribute_value``


## 4.2. Data
- observed 특성이 True 로 셋팅이 된 경우
- 값을 도중에 바꿀 수 없으며, fitting method (다음 장)에서 사용할 때 샘플링 되지 않습니다. 


### 4.2.1. Declaring stochastic variables to be data,  데이타 타입으로 선언하기

 observed=True 를 넣어주거나, @pymc.observed 처럼 decoration 을 이용합니다.

In [ ]:
@pymc.stochastic(observed=True)

x = pymc.Binomial('x', n=n, p=p, observed=True)

In [ ]:
@pymc.observed(dtype=int)
def ...

## 4.3 The Deterministic class

 부모에 의해 완전히 그 값이 정해진 변수들을 의미합니다. 예를 들면, 재난 발생 비율인 rate는 switing point와  early_mean, late_mean 에 의해 그 값이 완전 고정되는 변수입니다.   

rt= e t≤s 
    l t>s,



### 4.3.1. Creation of deterministic variables
stochastic variables과 비슷하게, automatic, decorator, and direct interfaces로 정의 가능합니다. 

1) Automatic
A handful of common functions have been wrapped in Deterministic objects. These are brief enough to list:

LinearCombination: 벡터의 내적곱(? inner product)를 계산합니다

Index:
Has two parents x and index. x must be iterables, index must be valued as an integer. The value of an index is: x[𝚒𝚗𝚍𝚎𝚡]Ty[𝚒𝚗𝚍𝚎𝚡]. Index is useful for implementing dynamic models, in which the parent-child connections change.

Lambda:
Converts an anonymous function (in Python, called lambda functions) to a Deterministic instance on a single line.

CompletedDirichlet:
PyMC represents Dirichlet variables of length k by the first k−1 elements; since they must sum to 1, the kth element is determined by the others. CompletedDirichlet appends the kth element to the value of its parent D.

Logit, InvLogit, StukelLogit, StukelInvLogit:
Common link functions for generalized linear models, and their inverses.
It’s a good idea to use these classes when feasible in order to give hints to step methods.

Elementary operations on variables
Certain elementary operations on variables create deterministic variables. For example:


2) Decorator
A deterministic variable can be created via a decorator in a way very similar to Stochastic‘s decorator interface:

In [ ]:
@deterministic(plot=False)
def rate(s=switchpoint, e=early_mean, l=late_mean):
    ''' Concatenate Poisson means '''
    out = empty(len(disasters_array))
    out[:s] = e
    out[s:] = l
    return out

Stochastic과 달리 로그가 필요 없고 부모로 상속받은 값을 연산해서 가집니다. 

3) Direct
Deterministic objects can also be instantiated directly:

In [ ]:
def rate_eval(switchpoint = s, early_rate = e, late_rate = l):
    value = zeros(N)
    value[:switchpoint] = early_rate
    value[switchpoint:] = late_rate
    return value

rate = pymc.Deterministic(eval = rate_eval,
                  name = 'rate',
                  parents = {'switchpoint': switchpoint,
                          'early_mean': early_mean,
                          'late_mean': late_mean}),
                  doc = 'The rate of disaster occurrence.',
                  trace = True,
                  verbose = 0,
                  dtype=float,
                  plot=False,
                  cache_depth = 2)

## 4.4. Containers

Here, y depends on every element of the Markov chain x, but we wouldn’t want to manually enter N parent labels `x_0', `x_1', etc.

This situation can be handled naturally in PyMC:

In [ ]:
N = 10
x_0 = pymc.Normal(`x_0', mu=0, tau=1)

x = np.empty(N, dtype=object)
x[0] = x_0

for i in range(1, N):

    x[i] = pymc.Normal(`x_%i' % i, mu=x[i-1], tau=1)

@pymc.observed
def y(value=1, mu=x, tau=100):
    return pymc.normal_like(value, np.sum(mu**2), tau)

 PyMC 는 자동적으로 Container class로 array x를 처리합니다....(?) 
 Containers는 회귀문의 일종으로 container 함수 내의 변수들을 iteration number 에 맞게 변수/오브젝트를 정의해주는 것으로, lists, tuples, dictionaries, Numpy arrays, modules, sets or any object with a __dict__ attribute에 해당하는 객체?들을 처리합니다. 변수 혹은 변수가 아닌 타입들도 자유롭게 섞일 수 있고 다른 타이의 컨테이너를 종속(nest?)할 수 있습니다.  Containers attempt to behave like the objects they wrap. All containers are subclasses of ContainerBase.
 

## 4.5. The Potential class
The joint density corresponding to model ((?)) can be written as follows:

p(D,s,l,e)=p(D|s,l,e)p(s)p(l)p(e).

예를 들면 p(D|s,l,e), p(s), p(l) 또는 p(e) 의 경우는 Stochastic object라고 볼 수 있을 것이다. 
하지만 예를 들어 다음과 같이 joint density를 바꾸고 싶은 경우에, 

p(D,s,l,e)∝p(D|s,l,e)p(s)p(l)p(e)I(|e−l|<1)


 class Potential에서 수정이 가능하다.  Markov random field 와 같은 hierarchy에 대한 정의도 가능. 


4.5.1. An example of soft data; 왜 포텐셜이 필요할까? 

p(t|S(x,β))p(β)

to

γ(S)p(t|S(x,β))p(β),

임의값인 γ(S) 보다는 priors on S and β 에 대한 fitting the model을 이용해 data t를 지울 수 있습니다.

Its important to understand that γ is not a variable, so it does not have a value. That means, among other things, there will be no γ column in MCMC traces....?



### 4.5.2. Creation of Potentials
There are two ways to create potentials:

Decorator
A potential can be created via a decorator in a way very similar to Deterministic‘s decorator interface:

In [ ]:
@pymc.potential
def psi_i(x_lo = x[i], x_hi = x[i+1]):
    """A pair potential"""
    return -(xlo - xhi)**2


In [ ]:
def psi_i_logp(x_lo = x[i], x_hi = x[i+1]):
    return -(xlo - xhi)**2

psi_i = pymc.Potential(logp = psi_i_logp,
                    name = 'psi_i',
                    parents = {'xlo': x[i], 'xhi': x[i+1]},
                    doc = 'A pair potential',
                    verbose = 0,
                    cache_depth = 2)